In [5]:
from math import log
import random
import numpy as np
from scipy.linalg import eig 

space = [1,2,3,4]
px1 = [[0.5,0.25,0.125,0.125]]
n = 10 #總共十次
#x = [1, 1, 3, 3, 3, 4, 1, 2, 4, 1]#給一組數組由不同次數的1,2,3,4組成
P = [[0.5,0.25,0.125,0.125],[0.25,0.125,0.125,0.5],[0.125,0.125,0.5,0.25],[0.25,0.125,0.125,0.5]]

def Markov_chain(space, px1, P, n): #Write an Markov chain function
        CurrentState = space[random.randint(0,3)]#隨機一個狀態成為當前的狀態
        Markov_Results = []
        for i in range (0,n):
            FutureState = np.random.choice(space, p=P[CurrentState-1]) #未來狀態的產生是從一開始隨機產生的目前狀態所屬的機率列擇出
            CurrentState = FutureState
            Markov_Results.append(FutureState)
        return Markov_Results

x = Markov_chain(space,px1,P,n)
#print(x)

Key_frequences = [] #給一定空陣列儲存1,2,3,4分別出現幾次
ones = 0
twos = 0
threes = 0
fours = 0
for i in range (0,n):#寫個for判斷1,2,3,4各出現幾次
    if x[i] == 1:
        ones = ones+1
    if x[i] == 2:
        twos = twos+1
    if x[i] == 3:
        threes = threes+1
    if x[i] == 4:
        fours = fours+1
Key_frequences = [('1',ones), ('2',twos) , ('3',threes) , ('4',fours)]
Key_frequences.sort(key=lambda Key_frequences: Key_frequences[1]) #將數組有小排到大,排列順序的參考由Key_frequences[1]決定
#print(Key_frequences[0][1])

class Node: #建立一個物件來初始化節點 space用來表狀態, freq表各個狀態出現的次數, left表在節點左邊(小), right表在節點右邊(大)
    def __init__(self,space, freq):
        self.space = space
        self.freq = freq
        self.value = '0'
        self.left = None
        self.right = None

#開始把排列好的數組存成各個節點
nodes = []
for i in range (0,len(Key_frequences)):
    #Key_frequences[i][0]表示key , Key_frequences[i][1]表示出現次數    
    nodes.append(Node(Key_frequences[i][0],Key_frequences[i][1])) 
#print(nodes[0]) # [key] , 出現次數]

#開始建立Huffman樹
while(len(nodes)>1):#while會一直跑,直到nodes剩下一個就是Root, leaf->middle->Root
    #建立新節點時,不賦予新節點狀態 Node (space=None, 加總次數) 因為前面有排序,所以我知道左邊開始最小,將左邊小的兩個次數相加
    node_new = Node(None, nodes[0].freq+nodes[1].freq)
    node_new.left = nodes[0]#指定最左邊(最小)的放在新節點的左下方
    node_new.right = nodes[1]#指定倒數第二左邊(倒數第二小)的放在新節點的右下方
    del nodes[0]#移除最左邊
    del nodes[0]#再移除一次
    nodes.append(node_new)#將新的節點放回到原本的陣列裡面
    nodes.sort(key=lambda node_new:node_new.freq)#重新做排序,依然用次數來由小到大排列
#找到最上面的節點,就是Root
root = nodes[0]
#print(root.space)#給的是記憶體位址,所以是指標類型

#這裡好Tricky
#print(root.right.right.right.space)#依照前面我給的x數組可以知道狀態'4'的出現頻率倒數第二低,因此從Root的右邊開始往下爬到葉節點為4

codes = {}#建立一個容器儲存各個key所代表的編碼
reverse_mapping = {}#驗證用

####網路找到的,還看不懂 這裡也很Tricky
#開始爬樹建立各個key所代表的編碼, 用遞迴方法 Recursion
def make_codes_helper(root, code):
    if(root == None):#防呆用
        return
    if(root.space != None):#Root節點必定是None所以第一次跑不近來這裡
        codes[root.space] = code
        reverse_mapping[code] = root.space
        return
    #以下是當root.space是None時,如果遇到左邊就加零,如果遇到右邊就加一
    # root[0].space == None -x if(root.space != None) -> root[0].left
    make_codes_helper(root.left, code+"0")
    make_codes_helper(root.right, code+"1")

""" while 還沒走完二元樹
    先看目前右邊有沒有路(有喔)-> 存下來
    在看左邊有沒有路(有喔)->往左走
    左邊沒路了->最後一個記下來的右邊的路 """


#print(code)
code = ""
make_codes_helper(root,code) 

#我知道第一個root的space肯定是null,第二個有可能是null,第三個就到葉節點了,所以我要判斷在走每一個節點時首先 
#.space是不是None? 如果是,就往左下(+0)與右下(+1)走,直到 .space有值時就是葉節點了

#開始Encoding
encode_text = ""
for character in x:
    encode_text += codes[str(character)]
#開始Encoding

#找出各狀態的機率
px = []
for i in range (len(Key_frequences)):
    px.append(Key_frequences[i][1]/10.0) 
#找出各狀態的機率

#找出各狀態所擁有的編碼長度
c_words1 = codes.get('1')
c_words2 = codes.get('2')
c_words3 = codes.get('3')
c_words4 = codes.get('4')
length_1 = len(c_words1)
length_2 = len(c_words2)
length_3 = len(c_words3)
length_4 = len(c_words4)
length = [length_1, length_2, length_3, length_4]
length.sort(reverse = True)
#找出各狀態所擁有的編碼長度

#求出Code Rate
expect_length = 0
for i in range (len(length)):
    expect_length  += px [i] * length[i]
R = expect_length/10.0
#求出Code Rate

#求出Entropy Rate
freqs = []
for i in range (len(Key_frequences)):
    freqs.append(Key_frequences[i][1])
Entropy_Rate = 0
for i in range (len(Key_frequences)):
    Entropy_Rate += -(freqs[i]*px[i]*log(px[i],2)/10)
#求出Entropy Rate

#開始Decoding
current_code=""
decoded_text=""
for bit in encode_text:
    current_code+=bit
    if(current_code in reverse_mapping):
        character = reverse_mapping[current_code]
        decoded_text += character
        current_code = ""
#開始Decoding

###網路找到的,還看不懂
#求出 Stationary Distribution
transition_mat = np.matrix([
    [0.5,0.25,0.125,0.125],\
    [0.25,0.125,0.125,0.5],\
    [0.125,0.125,0.5,0.25],\
    [0.25,0.125,0.125,0.5]])
S, U = eig(transition_mat.T)
stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
stationary_z = stationary / np.sum(stationary)
#求出 Stationary Distribution

print(x)
print(codes)
print(R)
print(Entropy_Rate)
print(stationary_z)
print(decoded_text)
#print(root.right.right.space)

[1, 1, 2, 4, 4, 2, 2, 4, 1, 3]
{'3': '00', '1': '01', '2': '10', '4': '11'}
0.2
0.5021999913737492
[0.3    0.1625 0.2    0.3375]
1124422413
